## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path_s = path.join(getcwd(), "TrustZone-S")
project_path_s

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\TrustZone\\TrustZone-S'

In [6]:
project_path_ns = path.join(getcwd(), "TrustZone-NS")
project_path_ns

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\TrustZone\\TrustZone-NS'

In [7]:
atprogram(project_path_ns, verbose=2)
atprogram(project_path_s, verbose=2, erase=False)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:5048
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00008000. Size = 0x000006b4.
[DEBUG] Memory segment base written at 0x000086b4. Size = 0x00000064.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "program"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:5050
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Memory segment base written at 

0

## Data Logging

Create a figure for the plot.

In [8]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [True, True, True, True],
    "gpio_delay_time" : 0.0007,
    "plot_pins_method": "line",
    "plot_xmax": 100,
    "fig": fig,
    "window_title": "Experiment TrustZone",
    "file_name_base": "experiment_trustzone"
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
fig.clf()
for ax in fig.get_axes():
    ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(config_dict["plot_xmax"],stop_fn)
    data = dgilib.data

In [12]:
print(data)

Interfaces:
	 256:   power,   samples:   69000
	  48:   gpio,    samples:     804



## Analysis

In [13]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, 0.01)

In [14]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, 0.01)

In [15]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 100


In [16]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [17]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 1.1843e-14
    reduced chi-square = 1.2085e-16
    Akaike info crit   = -3663.22004
    Bayesian info crit = -3658.00970
[[Variables]]
    slope:      8.8247e-10 +/- 3.8083e-11 (4.32%) (init = 0)
    intercept:  2.4254e-08 +/- 2.2152e-09 (9.13%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 9.1575e-15
    reduced chi-square = 9.3444e-17
    Akaike info crit   = -3688.93718
    Bayesian info crit = -3683.72684
[[Variables]]
    slope:      9.0550e-10 +/- 3.3488e-11 (3.70%) (init = 0)
    intercept:  2.0915e-08 +/- 1.9479e-09 (9.31%) (init = 1)
[[Correlations]] (unrepor

In [18]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis TrustZone")

In [19]:
charge_color = 'r'
time_color = 'b'

In [20]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [21]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Store Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Load Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [22]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(nsc_store_charge, open("trustzone_store_charge.p", "wb"))
pickle.dump(nsc_load_charge, open("trustzone_load_charge.p", "wb"))